# LR Filtered Dataset

Testing the classifier when using a filtered dataset (selected malware types e.g. virus, trojan, worm)

In [2]:
import pandas as pd
import numpy as np
import math
from IPython.display import display
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
import lib.helpers as jcfg_helpers


samples = pd.read_csv('data/pe32_imports_labeled.csv')
samples['date'] = pd.to_datetime(samples['date'], format='%Y/%m/%d')
samples = samples.set_index('date').sort_index()

goodware = samples[samples.malware == 0]
malware = samples[samples.malware == 1]

print('Total goodware: {}'.format(len(goodware)))
print('Total malware: {}'.format(len(malware)))

Total goodware: 34509
Total malware: 67873


## Dataset for Trojans

In [2]:
microsoft_class = pd.read_csv('data/microsoft_classification.csv')
microsoft_class.fillna('', inplace=True)
symantec_class = pd.read_csv('data/symantec_classification.csv')
symantec_class.fillna('', inplace=True)
trendmicro_class = pd.read_csv('data/trendmicro_classification.csv')
trendmicro_class.fillna('', inplace=True)

microsoft_filter = microsoft_class[microsoft_class.type == 'trojan']
symantec_filter = symantec_class[symantec_class.prefix == 'trojan']
trendmicro_filter = trendmicro_class[trendmicro_class.prefix == 'troj']

links = pd.concat([microsoft_filter.link, symantec_filter.link, trendmicro_filter.link]).unique()

filtered_malware = samples[samples.link.isin(links)]

print('Total malware: {}'.format(len(filtered_malware)))

Total malware: 41870


## Testing

Creating multiple LR and varying the ratio of malware/goodware in training/validation

In [12]:
for i in np.arange(0.1, 1.1, 0.1):
    print('\t\tDownscale of {:.1f}'.format(i))
    # Training/Validation creation
    training, validation = jcfg_helpers.train_test_split(
    pd.concat([goodware, filtered_malware]),
    validation_size = 0.25,
    balanced = True,
    malware_downscale = i)
    #print('Malware in training: {}\nGoodware in training: {}'.format(len(training[training.malware == 1]),
    #                                                                 len(training[training.malware == 0])))
    #print('Malware in validation: {}\nGoodware in validation: {}'.format(len(validation[validation.malware == 1]),
    #                                                                     len(validation[validation.malware == 0])))
    
    ################
    # Feature Vector
    cv_token_pattern = u'[^;]+'
    min_df = 2
    cv = CountVectorizer(token_pattern=cv_token_pattern, min_df=min_df)
    train_X = cv.fit_transform(training.imports)
    #print('Total features: {}'.format(len(cv.get_feature_names())))
    
    ################
    # Logistic Regression
    train_Y = training.malware
    lr = LogisticRegression().fit(train_X, train_Y)

    ################
    # Scoring
    test_X = cv.transform(validation.imports)
    test_Y = validation.malware
    score = confusion_matrix(test_Y, lr.predict(test_X))
    fp_rate, fn_rate, cc_rate, det_rate = jcfg_helpers.calc_ratios(score)
    print('CC:\t{:.4f}'.format(cc_rate))
    print('DR:\t{:.4f}'.format(det_rate))
    print('FP:\t{:.4f}'.format(fp_rate))
    print('FN:\t{:.4f}'.format(fn_rate))
    malware_only = lr.predict_proba(test_X[np.where(test_Y == 1)])
    goodware_only = lr.predict_proba(test_X[np.where(test_Y == 0)])
    print('\nMalware stats:')
    display(pd.Series(malware_only[:,1]).describe())
    print('\nGoodware stats:')
    display(pd.Series(goodware_only[:,0]).describe())
    print()

		Downscale of 0.1
CC:	0.9150
DR:	0.3082
FP:	0.4426
FN:	0.0661

Malware stats:


count    3.310000e+02
mean     3.633775e-01
std      3.564969e-01
min      2.154350e-07
25%      5.470049e-02
50%      2.420007e-01
75%      6.038979e-01
max      9.999999e-01
dtype: float64


Goodware stats:


count    3318.000000
mean        0.942601
std         0.132651
min         0.024514
25%         0.945300
50%         0.989109
75%         0.999479
max         1.000000
dtype: float64


		Downscale of 0.2
CC:	0.8963
DR:	0.5415
FP:	0.2329
FN:	0.0865

Malware stats:


...

count    3.318000e+03
mean     7.731386e-01
std      2.750734e-01
min      2.318087e-08
25%      4.887798e-01
50%      9.282055e-01
75%      9.980447e-01
max      1.000000e+00
dtype: float64


		Downscale of 1.0
CC:	0.7819
DR:	0.8677
FP:	0.2593
FN:	0.1597

Malware stats:


count    3.318000e+03
mean     7.214340e-01
std      2.671056e-01
min      2.697429e-10
25%      5.520293e-01
50%      8.297241e-01
75%      9.601650e-01
max      1.000000e+00
dtype: float64


Goodware stats:


count    3.318000e+03
mean     7.575619e-01
std      2.871125e-01
min      1.954325e-08
25%      4.479707e-01
50%      9.216381e-01
75%      9.974889e-01
max      1.000000e+00
dtype: float64

## Dataset for Backdoors

In [20]:
microsoft_class = pd.read_csv('data/microsoft_classification.csv')
microsoft_class.fillna('', inplace=True)
symantec_class = pd.read_csv('data/symantec_classification.csv')
symantec_class.fillna('', inplace=True)
trendmicro_class = pd.read_csv('data/trendmicro_classification.csv')
trendmicro_class.fillna('', inplace=True)

microsoft_filter = microsoft_class[microsoft_class.type == 'backdoor']
symantec_filter = symantec_class[symantec_class.prefix == 'backdoor']
trendmicro_filter = trendmicro_class[trendmicro_class.prefix == 'bkdr']

links = pd.concat([microsoft_filter.link, symantec_filter.link, trendmicro_filter.link]).unique()

filtered_malware = samples[samples.link.isin(links)]

print('Total malware: {}'.format(len(filtered_malware)))

Total malware: 13273


## Testing

Creating multiple LR and varying the ratio of malware/goodware in training/validation

In [9]:
for i in np.arange(0.1, 1.1, 0.1):
    print('\t\tDownscale of {:.1f}'.format(i))
    # Training/Validation creation
    training, validation = jcfg_helpers.train_test_split(
    pd.concat([goodware, filtered_malware]),
    validation_size = 0.25,
    balanced = True,
    malware_downscale = i)
    #print('Malware in training: {}\nGoodware in training: {}'.format(len(training[training.malware == 1]),
    #                                                                 len(training[training.malware == 0])))
    #print('Malware in validation: {}\nGoodware in validation: {}'.format(len(validation[validation.malware == 1]),
    #                                                                     len(validation[validation.malware == 0])))
    
    ################
    # Feature Vector
    cv_token_pattern = u'[^;]+'
    min_df = 2
    cv = CountVectorizer(token_pattern=cv_token_pattern, min_df=min_df)
    train_X = cv.fit_transform(training.imports)
    #print('Total features: {}'.format(len(cv.get_feature_names())))
    
    ################
    # Logistic Regression
    train_Y = training.malware
    lr = LogisticRegression().fit(train_X, train_Y)

    ################
    # Scoring
    test_X = cv.transform(validation.imports)
    test_Y = validation.malware
    score = confusion_matrix(test_Y, lr.predict(test_X))
    fp_rate, fn_rate, cc_rate, det_rate = jcfg_helpers.calc_ratios(score)
    print('CC:\t{:.4f}'.format(cc_rate))
    print('DR:\t{:.4f}'.format(det_rate))
    print('FP:\t{:.4f}'.format(fp_rate))
    print('FN:\t{:.4f}'.format(fn_rate))
    malware_only = lr.predict_proba(test_X[np.where(test_Y == 1)])
    goodware_only = lr.predict_proba(test_X[np.where(test_Y == 0)])
    print('\nMalware stats:')
    display(pd.Series(malware_only[:,1]).describe())
    print('\nGoodware stats:')
    display(pd.Series(goodware_only[:,0]).describe())
    print()

		Downscale of 0.1
CC:	0.9150
DR:	0.3082
FP:	0.4426
FN:	0.0661

Malware stats:


count    3.310000e+02
mean     3.633775e-01
std      3.564969e-01
min      2.154350e-07
25%      5.470049e-02
50%      2.420007e-01
75%      6.038979e-01
max      9.999999e-01
dtype: float64


Goodware stats:


count    3318.000000
mean        0.942601
std         0.132651
min         0.024514
25%         0.945300
50%         0.989109
75%         0.999479
max         1.000000
dtype: float64


		Downscale of 0.2
CC:	0.8963
DR:	0.5415
FP:	0.2329
FN:	0.0865

Malware stats:


...

count    3.318000e+03
mean     7.731386e-01
std      2.750734e-01
min      2.318087e-08
25%      4.887798e-01
50%      9.282055e-01
75%      9.980447e-01
max      1.000000e+00
dtype: float64


		Downscale of 1.0
CC:	0.7819
DR:	0.8677
FP:	0.2593
FN:	0.1597

Malware stats:


count    3.318000e+03
mean     7.214340e-01
std      2.671056e-01
min      2.697429e-10
25%      5.520293e-01
50%      8.297241e-01
75%      9.601650e-01
max      1.000000e+00
dtype: float64


Goodware stats:


count    3.318000e+03
mean     7.575619e-01
std      2.871125e-01
min      1.954325e-08
25%      4.479707e-01
50%      9.216381e-01
75%      9.974889e-01
max      1.000000e+00
dtype: float64

## Dataset for Worms

In [35]:
microsoft_class = pd.read_csv('data/microsoft_classification.csv')
microsoft_class.fillna('', inplace=True)
symantec_class = pd.read_csv('data/symantec_classification.csv')
symantec_class.fillna('', inplace=True)
trendmicro_class = pd.read_csv('data/trendmicro_classification.csv')
trendmicro_class.fillna('', inplace=True)

microsoft_filter = microsoft_class[microsoft_class.type == 'worm']
symantec_filter = symantec_class[symantec_class.suffix == 'worm']
trendmicro_filter = trendmicro_class[trendmicro_class.prefix == 'worm']

l = [i for i in microsoft_filter.link if i not in trendmicro_filter.link.values]
display(len(l))

links = pd.Series(l)

# inter = microsoft_filter.link.isin([symantec_filter.link, trendmicro_filter])
#inter = pd.Index([microsoft_filter.link.values, symantec_filter.link.values, trendmicro_filter.link.values])
# links = pd.concat(inter)
# links = pd.concat([microsoft_filter.link, symantec_filter.link, trendmicro_filter.link]).unique()
# display(inter[0])

#links = pd.concat(pd.Series([inter[0], inter[1], inter[2]]))

filtered_malware = samples[samples.link.isin(links)]

print('Total malware: {}'.format(len(filtered_malware)))

6162

Total malware: 3800


In [36]:
display(trendmicro_class[trendmicro_class.link.isin(links)])

,date,md5,link,family,prefix,suffix
12,2013-04-16,452e20bda5be890364fc3a9905278b1d,YmRhMmY4YjBkNTM4NDllMTkyZjQzODg3N2ZmNTQwNDE,generic,pak,001
31,2013-04-17,8a41cd0a312259a1996e29c8609fa11b,MzhmNGJkYzc3NWZiNGUwNzlmODNhNDZiMGJhNTJiMWY,spnr,troj,11cp13
36,2013-04-17,f0063edef419009b1564e5fba81157db,Mjk3NmYwOTI4Y2EzNDcxMTk2YjBjNGI2OTA1MDZjOTY,fareit,tspy,tpa
38,2013-04-17,e332e11cdafb7683fc186245460a5f2a,NmVhYTQ3OGM2YTc0NDgyOGIxODcwZjJjMzg3YjZjNzA,gen,troj,rcbochr
51,2013-04-18,8a41cd0a312259a1996e29c8609fa11b,NzI3MGE2YmNlY2MzNGQ3ZWFhZGYxYTliODgyMzE0MGE,spnr,troj,11cp13
57,2013-04-18,f5c927945db809946363775a1ee3992d,ZDk0ZDFjY2QwZTI5NDU1OTg5MDBiMzhlMDIxYTljZjk,spnr,troj,02l111
68,2013-04-18,77591ef66f98d6c7db7892b81b131530,YzA1NzIxMzQ2OTlhNGYwNjlmMDc1Yjg0MWEyOTc2NGI,spnr,troj,0bda13
92,2013-04-19,19c70de08fbb2ce178c25d4afccc7397,NDUyYTdiOWMwZGM1NDg4OGEwODhiYWViODhhYjM3NWI,spnr,troj,02k811
96,2013-04-20,241bb8a2ce3461c1edcf59801b0ef427,NmI0MWFhZmQyNGM5NDRlNmJjYTViYTZhNjEzMTE4MmM,spnr,troj,11j212
99,2013-04-20,9352259f72b8ef603ab644de70bc09c4,OWM0NWIwMjkzMGZhNDFlYTg4NDg0MzRiZGMyNTE3MDU,gen,troj,r27cccl


## Testing

Creating multiple LR and varying the ratio of malware/goodware in training/validation

In [25]:
for i in np.arange(0.1, 1.1, 0.1):
    print('\t\tDownscale of {:.1f}'.format(i))
    # Training/Validation creation
    training, validation = jcfg_helpers.train_test_split(
    pd.concat([goodware, filtered_malware]),
    validation_size = 0.25,
    balanced = True,
    malware_downscale = i)
    #print('Malware in training: {}\nGoodware in training: {}'.format(len(training[training.malware == 1]),
    #                                                                 len(training[training.malware == 0])))
    #print('Malware in validation: {}\nGoodware in validation: {}'.format(len(validation[validation.malware == 1]),
    #                                                                     len(validation[validation.malware == 0])))
    
    ################
    # Feature Vector
    cv_token_pattern = u'[^;]+'
    min_df = 2
    cv = CountVectorizer(token_pattern=cv_token_pattern, min_df=min_df)
    train_X = cv.fit_transform(training.imports)
    #print('Total features: {}'.format(len(cv.get_feature_names())))
    
    ################
    # Logistic Regression
    train_Y = training.malware
    lr = LogisticRegression().fit(train_X, train_Y)

    ################
    # Scoring
    test_X = cv.transform(validation.imports)
    test_Y = validation.malware
    score = confusion_matrix(test_Y, lr.predict(test_X))
    fp_rate, fn_rate, cc_rate, det_rate = jcfg_helpers.calc_ratios(score)
    print('CC:\t{:.4f}'.format(cc_rate))
    print('DR:\t{:.4f}'.format(det_rate))
    print('FP:\t{:.4f}'.format(fp_rate))
    print('FN:\t{:.4f}'.format(fn_rate))
    malware_only = lr.predict_proba(test_X[np.where(test_Y == 1)])
    goodware_only = lr.predict_proba(test_X[np.where(test_Y == 0)])
    print('\nMalware stats:')
    display(pd.Series(malware_only[:,1]).describe())
    print('\nGoodware stats:')
    display(pd.Series(goodware_only[:,0]).describe())
    print()

		Downscale of 0.1
CC:	0.9733
DR:	0.9670
FP:	0.2143
FN:	0.0033

Malware stats:


count    91.000000
mean      0.923375
std       0.125254
min       0.343109
25%       0.911687
50%       0.961884
75%       0.999242
max       0.999976
dtype: float64


Goodware stats:


count    9.190000e+02
mean     9.551196e-01
std      1.291835e-01
min      4.145330e-07
25%      9.889712e-01
50%      9.986451e-01
75%      9.999345e-01
max      1.000000e+00
dtype: float64


		Downscale of 0.2
CC:	0.9728
DR:	0.9672
FP:	0.1194
FN:	0.0067

Malware stats:


...

count    9.190000e+02
mean     9.198976e-01
std      2.008439e-01
min      9.149481e-09
25%      9.840403e-01
50%      9.967235e-01
75%      9.999305e-01
max      1.000000e+00
dtype: float64


		Downscale of 1.0
CC:	0.9173
DR:	0.9064
FP:	0.0734
FN:	0.0916

Malware stats:


count    9.190000e+02
mean     8.527681e-01
std      2.500917e-01
min      4.674451e-07
25%      8.359314e-01
50%      9.682761e-01
75%      9.892080e-01
max      1.000000e+00
dtype: float64


Goodware stats:


count    9.190000e+02
mean     9.181464e-01
std      2.040309e-01
min      1.733618e-08
25%      9.839213e-01
50%      9.963814e-01
75%      9.999386e-01
max      1.000000e+00
dtype: float64

## Dataset for Virus

In [18]:
microsoft_class = pd.read_csv('data/microsoft_classification.csv')
microsoft_class.fillna('', inplace=True)
symantec_class = pd.read_csv('data/symantec_classification.csv')
symantec_class.fillna('', inplace=True)
trendmicro_class = pd.read_csv('data/trendmicro_classification.csv')
trendmicro_class.fillna('', inplace=True)

microsoft_filter = microsoft_class[microsoft_class.type == 'virus']
symantec_filter = symantec_class[symantec_class.prefix == 'virusdoctor']
trendmicro_filter = trendmicro_class[trendmicro_class.family == 'virus']

links = pd.concat([microsoft_filter.link, symantec_filter.link, trendmicro_filter.link]).unique()

filtered_malware = samples[samples.link.isin(links)]

print('Total malware: {}'.format(len(filtered_malware)))

Total malware: 4467


## Testing

Creating multiple LR and varying the ratio of malware/goodware in training/validation

In [19]:
for i in np.arange(0.1, 1.1, 0.1):
    print('\t\tDownscale of {:.1f}'.format(i))
    # Training/Validation creation
    training, validation = jcfg_helpers.train_test_split(
    pd.concat([goodware, filtered_malware]),
    validation_size = 0.25,
    balanced = True,
    malware_downscale = i)
    #print('Malware in training: {}\nGoodware in training: {}'.format(len(training[training.malware == 1]),
    #                                                                 len(training[training.malware == 0])))
    #print('Malware in validation: {}\nGoodware in validation: {}'.format(len(validation[validation.malware == 1]),
    #                                                                     len(validation[validation.malware == 0])))
    
    ################
    # Feature Vector
    cv_token_pattern = u'[^;]+'
    min_df = 2
    cv = CountVectorizer(token_pattern=cv_token_pattern, min_df=min_df)
    train_X = cv.fit_transform(training.imports)
    #print('Total features: {}'.format(len(cv.get_feature_names())))
    
    ################
    # Logistic Regression
    train_Y = training.malware
    lr = LogisticRegression().fit(train_X, train_Y)

    ################
    # Scoring
    test_X = cv.transform(validation.imports)
    test_Y = validation.malware
    score = confusion_matrix(test_Y, lr.predict(test_X))
    fp_rate, fn_rate, cc_rate, det_rate = jcfg_helpers.calc_ratios(score)
    print('CC:\t{:.4f}'.format(cc_rate))
    print('DR:\t{:.4f}'.format(det_rate))
    print('FP:\t{:.4f}'.format(fp_rate))
    print('FN:\t{:.4f}'.format(fn_rate))
    malware_only = lr.predict_proba(test_X[np.where(test_Y == 1)])
    goodware_only = lr.predict_proba(test_X[np.where(test_Y == 0)])
    print('\nMalware stats:')
    display(pd.Series(malware_only[:,1]).describe())
    print('\nGoodware stats:')
    display(pd.Series(goodware_only[:,0]).describe())
    print()

		Downscale of 0.1
CC:	0.8998
DR:	0.2432
FP:	0.5909
FN:	0.0724

Malware stats:


count    111.000000
mean       0.277960
std        0.294748
min        0.000009
25%        0.016794
50%        0.199848
75%        0.466603
max        0.940911
dtype: float64


Goodware stats:


count    1116.000000
mean        0.928899
std         0.151273
min         0.006975
25%         0.945831
50%         0.987215
75%         0.996882
max         1.000000
dtype: float64


		Downscale of 0.2
CC:	0.8656
DR:	0.5202
FP:	0.3862
FN:	0.0930

Malware stats:


...

count    1116.000000
mean        0.766688
std         0.302338
min         0.000190
25%         0.661756
50%         0.924666
75%         0.973224
max         1.000000
dtype: float64


		Downscale of 1.0
CC:	0.7836
DR:	0.7894
FP:	0.2197
FN:	0.2131

Malware stats:


count    1.116000e+03
mean     7.321270e-01
std      3.007564e-01
min      3.605102e-07
25%      5.908487e-01
50%      8.649940e-01
75%      9.676963e-01
max      1.000000e+00
dtype: float64


Goodware stats:


count    1116.000000
mean        0.746229
std         0.310891
min         0.000042
25%         0.616377
50%         0.895339
75%         0.969030
max         1.000000
dtype: float64